In [1]:
import pandas as pd
import numpy as np

In [5]:
from datetime import datetime
from pandas import Series

df_train_X = pd.read_csv('resources/X_train.csv')
df_test_X = pd.read_csv('resources/X_test.csv')
df_train_Y = pd.read_csv('resources/y_train.csv')

df = df_train_Y.merge(df_train_X)

df['salary_from'] = df['salary_from'].fillna(0)
df['salary_gross'] = df['salary_gross'].fillna(True)
df['description'] = df['description'].fillna("")

def get_month(text: Series):
    dt = datetime.strptime(text, '%Y-%m-%d %H:%M:%S+00')
    return dt.month

df['month'] = pd.DatetimeIndex(pd.to_datetime(df['created_at']).dt.date).month 
df_test_X['month'] = pd.DatetimeIndex(pd.to_datetime(df_test_X['created_at']).dt.date).month

df_test_X.head()

,id,name,has_test,response_letter_required,salary_from,salary_currency,salary_gross,published_at,created_at,employer_name,description,area_id,area_name,month
0,20978,Техник по инструменту / Специалист по инструме...,False,False,35000.0,RUR,False,2020-03-23 10:49:43+00,2020-03-23 10:49:43+00,АвтоСпецЦентр,<strong>Обязанности:</strong> <ul> <li>произво...,1,Москва,3
1,3102,Диспетчер по заказам,False,False,20000.0,RUR,False,2020-08-18 10:29:11+00,2020-08-18 10:29:11+00,АвтоВыкуп,<p>Приглашаем диспетчера по работе с заказами ...,1,Москва,8
2,14731,Frontend Developer в Focus Calendar,False,False,200000.0,RUR,False,2021-08-09 09:00:27+00,2021-08-09 09:00:27+00,Focus Calendar,<p><strong>Кого мы ищем и что тебе предстоит д...,1,Москва,8
3,16113,Системный аналитик,False,False,150000.0,RUR,False,2021-08-08 05:59:07+00,2021-08-08 05:59:07+00,INTECH,"<p>INTECH - IT-компания, ведущий контент и сер...",1,Москва,8
4,8410,Front-end разработчик (JavaScript),False,False,80000.0,RUR,False,2021-01-12 08:16:14+00,2021-01-12 08:16:14+00,Кравцов И.Н.,<p><strong>Требования к кандидату:</strong></p...,47,Кемерово,1


In [6]:
df['has_test'] = df['has_test'].replace({False: 0, True: 1})
df['response_letter_required'] = df['response_letter_required'].replace({False: 0, True: 1})
df['salary_gross'] = df['salary_gross'].replace({False: 0, True: 1})

df_test_X['has_test'] = df_test_X['has_test'].replace({False: 0, True: 1})
df_test_X['response_letter_required'] = df_test_X['response_letter_required'].replace({False: 0, True: 1})
df_test_X['salary_gross'] = df_test_X['salary_gross'].replace({False: 0, True: 1})

df_test_X.head()

,id,name,has_test,response_letter_required,salary_from,salary_currency,salary_gross,published_at,created_at,employer_name,description,area_id,area_name,month
0,20978,Техник по инструменту / Специалист по инструме...,0,0,35000.0,RUR,0.0,2020-03-23 10:49:43+00,2020-03-23 10:49:43+00,АвтоСпецЦентр,<strong>Обязанности:</strong> <ul> <li>произво...,1,Москва,3
1,3102,Диспетчер по заказам,0,0,20000.0,RUR,0.0,2020-08-18 10:29:11+00,2020-08-18 10:29:11+00,АвтоВыкуп,<p>Приглашаем диспетчера по работе с заказами ...,1,Москва,8
2,14731,Frontend Developer в Focus Calendar,0,0,200000.0,RUR,0.0,2021-08-09 09:00:27+00,2021-08-09 09:00:27+00,Focus Calendar,<p><strong>Кого мы ищем и что тебе предстоит д...,1,Москва,8
3,16113,Системный аналитик,0,0,150000.0,RUR,0.0,2021-08-08 05:59:07+00,2021-08-08 05:59:07+00,INTECH,"<p>INTECH - IT-компания, ведущий контент и сер...",1,Москва,8
4,8410,Front-end разработчик (JavaScript),0,0,80000.0,RUR,0.0,2021-01-12 08:16:14+00,2021-01-12 08:16:14+00,Кравцов И.Н.,<p><strong>Требования к кандидату:</strong></p...,47,Кемерово,1


In [7]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = BertModel.from_pretrained("cointegrated/rubert-tiny")

def embed_bert_cls(text):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

df['description_embeddings'] = df['description'].apply(embed_bert_cls)
df_test_X['description_embeddings'] = df_test_X['description'].apply(embed_bert_cls)

In [11]:
embedding_columns = [f'description_embedding_{i}' for i in range(df['description_embeddings'][0].shape[0])]
df[embedding_columns] = pd.DataFrame(df['description_embeddings'].to_list(), index=df.index)

embedding_columns = [f'description_embedding_{i}' for i in range(df_test_X['description_embeddings'][0].shape[0])]
df_test_X[embedding_columns] = pd.DataFrame(df_test_X['description_embeddings'].to_list(), index=df_test_X.index)   

df.head()

C:\Users\snoop\AppData\Local\Temp\ipykernel_10604\1674368537.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[embedding_columns] = pd.DataFrame(df['description_embeddings'].to_list(), index=df.index)
C:\Users\snoop\AppData\Local\Temp\ipykernel_10604\1674368537.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[embedding_columns] = pd.DataFrame(df['description_embeddings'].to_list(), index=df.index)
C:\Users\snoop\AppData\Local\Temp\ipykernel_10604\1674368537.py:2: PerformanceWarning: DataFrame is highly fragmented.  Th

,id,salary_to,name,has_test,response_letter_required,salary_from,salary_currency,salary_gross,published_at,created_at,...,description_embedding_302,description_embedding_303,description_embedding_304,description_embedding_305,description_embedding_306,description_embedding_307,description_embedding_308,description_embedding_309,description_embedding_310,description_embedding_311
0,29083,200000.0,Менеджер по продажам (брокер) элитной городско...,0,0,0.0,RUR,0,2020-06-16 06:45:04+00,2020-06-16 06:45:04+00,...,0.028885,-0.043466,-0.026413,0.027464,-0.044628,0.068238,-0.015694,-0.003241,0.114051,-0.008783
1,26052,170000.0,Product marketing manager / Продуктовый маркет...,0,0,120000.0,RUR,0,2020-07-22 10:24:15+00,2020-07-22 10:24:15+00,...,0.056030,-0.031484,-0.026959,0.049771,-0.026727,0.046751,0.002309,0.001899,0.050884,-0.019117
2,24055,120000.0,Программист Python (Middle),0,0,0.0,RUR,0,2020-04-12 08:19:08+00,2020-04-12 08:19:08+00,...,0.026668,-0.072043,-0.005280,0.002243,-0.037762,0.068617,-0.003391,-0.009008,0.112944,0.011726
3,4408,38000.0,"Продавец-кассир (Москва, Вернадского, 93)",0,0,33500.0,RUR,1,2020-08-24 01:16:52+00,2020-08-24 01:16:52+00,...,0.070586,0.011839,-0.017913,0.034098,-0.034372,0.028373,0.003799,-0.005530,0.140289,-0.000779
4,16856,35000.0,Дизайнер-визуализатор,0,1,30000.0,RUR,1,2021-09-18 13:30:15+00,2021-09-18 13:30:15+00,...,0.032004,-0.056066,-0.042197,0.022027,-0.059937,0.070181,-0.013968,0.022646,0.110343,0.002912


In [21]:
df_test_X[embedding_columns] = df_test_X[embedding_columns].apply(lambda x: x/100)
df[embedding_columns] = df[embedding_columns].apply(lambda x: x/100)

df.head()

,id,salary_to,name,has_test,response_letter_required,salary_from,salary_currency,salary_gross,published_at,created_at,...,description_embedding_302,description_embedding_303,description_embedding_304,description_embedding_305,description_embedding_306,description_embedding_307,description_embedding_308,description_embedding_309,description_embedding_310,description_embedding_311
0,29083,200000.0,Менеджер по продажам (брокер) элитной городско...,0,0,0.0,RUR,0,2020-06-16 06:45:04+00,2020-06-16 06:45:04+00,...,0.028885,-0.043466,-0.026413,0.027464,-0.044628,0.068238,-0.015694,-0.003241,0.114051,-0.008783
1,26052,170000.0,Product marketing manager / Продуктовый маркет...,0,0,120000.0,RUR,0,2020-07-22 10:24:15+00,2020-07-22 10:24:15+00,...,0.056030,-0.031484,-0.026959,0.049771,-0.026727,0.046751,0.002309,0.001899,0.050884,-0.019117
2,24055,120000.0,Программист Python (Middle),0,0,0.0,RUR,0,2020-04-12 08:19:08+00,2020-04-12 08:19:08+00,...,0.026668,-0.072043,-0.005280,0.002243,-0.037762,0.068617,-0.003391,-0.009008,0.112944,0.011726
3,4408,38000.0,"Продавец-кассир (Москва, Вернадского, 93)",0,0,33500.0,RUR,1,2020-08-24 01:16:52+00,2020-08-24 01:16:52+00,...,0.070586,0.011839,-0.017913,0.034098,-0.034372,0.028373,0.003799,-0.005530,0.140289,-0.000779
4,16856,35000.0,Дизайнер-визуализатор,0,1,30000.0,RUR,1,2021-09-18 13:30:15+00,2021-09-18 13:30:15+00,...,0.032004,-0.056066,-0.042197,0.022027,-0.059937,0.070181,-0.013968,0.022646,0.110343,0.002912


In [41]:
X = (['id', 'name', 'has_test', 'response_letter_required', 'salary_from', 'employer_name', 'area_name', 'description', 'created_at', 'month'] +
     [f'description_embedding_{i}' for i in range(df_test_X['description_embeddings'][0].shape[0])])
cat_features = ['name', 'employer_name', 'area_name', 'description', 'created_at']
y = ['salary_to']

# df = df.reindex(columns=X)
# df_test_X = df_test_X.reindex(columns=['month'])
# df.head()
# df_test_X.info()
# df_test_X.head()

In [49]:
df_copy = df.copy()
df_test_X_copy = df_test_X.copy()

In [50]:
t = df_copy.groupby('area_id')[['salary_to']].count().sort_values(by = 'salary_to')

areas_rare = np.array(t[t['salary_to'] < 10].index)

df_copy.drop(df.loc[df['area_id'].isin(areas_rare)].index, inplace = True)

In [51]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

train, test = train_test_split(df_copy[df_copy['salary_from'] < 500000], train_size=0.6, random_state=42)
val, test = train_test_split(test, train_size=0.5, random_state=42)

# X = ['name', 'has_test', 'response_letter_required', 'salary_from', 'employer_name', 'area_name']
# cat_features = ['name', 'employer_name', 'area_name']


model = CatBoostRegressor(
    cat_features=cat_features,
    eval_metric='SMAPE',
    learning_rate=0.065,
    random_seed=140,
    loss_function='MAE',
    verbose=15
)

model.fit(df_copy[X], df_copy[y])

test['salary_pred'] = model.predict(test[X])
predict = model.predict(df_test_X_copy[X])

def smape(y_true, y_pred):
    return 100 / len(y_true) * np.sum(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

submit_id = df_test_X_copy.id.to_list()
result = pd.DataFrame({'id': submit_id, 'salary_to': np.round(predict)})
result.to_csv('submission.csv', index=False)

print(smape(test['salary_pred'], test['salary_to']))


0:	learn: 52.5738576	total: 139ms	remaining: 2m 18s
15:	learn: 34.2637953	total: 1.45s	remaining: 1m 28s
30:	learn: 28.7808387	total: 2.56s	remaining: 1m 20s
45:	learn: 26.7593951	total: 3.45s	remaining: 1m 11s
60:	learn: 26.0678042	total: 4.43s	remaining: 1m 8s
75:	learn: 25.7096484	total: 5.46s	remaining: 1m 6s
90:	learn: 25.1538032	total: 6.46s	remaining: 1m 4s
105:	learn: 24.7981581	total: 7.48s	remaining: 1m 3s
120:	learn: 24.6165878	total: 8.56s	remaining: 1m 2s
135:	learn: 24.5089908	total: 9.6s	remaining: 1m
150:	learn: 24.3626115	total: 10.7s	remaining: 1m
165:	learn: 24.2208511	total: 11.8s	remaining: 59.2s
180:	learn: 24.0543708	total: 12.8s	remaining: 57.9s
195:	learn: 23.9082976	total: 13.8s	remaining: 56.4s
210:	learn: 23.7878978	total: 14.7s	remaining: 55s
225:	learn: 23.7183751	total: 15.7s	remaining: 53.8s
240:	learn: 23.6599971	total: 16.7s	remaining: 52.5s
255:	learn: 23.6038597	total: 17.6s	remaining: 51.3s
270:	learn: 23.5224719	total: 18.6s	remaining: 50.1s
285:	l